# Efficient Quantum Expectation Values Estimation for Concentrated States

## Summary




# 1. Introduction

The current generation of quantum processors is characterized by limited numbers of qubits, low gate fidelities, short coherence times, and noise processes that limit circuit depth. In this context, the Variational Quantum Algorithms (VQAs) is the best hope for obtaining quantum advantage. Those hybrid quantum-classical algorithms typically utilize quantum computers to measure expectation values of observables for a given state realized by a parametrized quantum circuit. However, precisely estimating these expectation values requires numerous measurement repetitions to reduce statistical fluctuation. Many applications require such high precision that VQA algorithms are impractical due to the enormous amount of time needed. For instance, quantum chemical calculations require the so-called chemical accuracy of $ 1 \textrm{ kcal/mol } \approx 1.6 \times 10^{-3}$ Hartree, making the Variational Quantum Eigensolver (VQE) method non-viable even for a small number of molecules.

In this project, we provide a Python package that implements a hybrid quantum-classical algorithm to measure the expectation values of observables efficiently. The algorithm is based on the paper "Quantum expectation-value estimation by computational basis sampling" and designed to be effective when the state is concentrated, i.e., when only a small number of amplitudes in a measurement basis are non-negligible. Several quantum states of practical importance are concentrated, and they often emerge  in quantum chemistry problems.

This tutorial starts with a brief explanation of the computation of the expectation values of observables. Next, we explain how to use the supplied package using instructive examples. Moreover, we assess its performance by measuring the resources required to achieve a fixed accuracy. Finally, we illustrate how to implement the VQE method for a molecular Hamiltonian.

## 2.  Measuring expectation values of observables

Let us consider a $n$-qubit system described by the pure state
\begin{align}
|\psi \rangle = \sum_{i=0}^{d-1} c_i |i \rangle,
\end{align}

where $i = i_{n-1} ... i_1 i_0$ is the integer representation of a binary string and $d = 2^n$ is the dimension of the system. The coefficients $c_i$ are complex numbers such that $\sum_{i=0}^{d-1} |c_i|^2 =1$. An observable $O$ acting on $d$ qubits can be expressed as a linear combination of products of Pauli and identity operators on single qubits by

\begin{align}
 O = \sum_{k=1}^M  \alpha_k  P_k,
\end{align}

where $P_k \in \{I, X,Y, Z\}^{\otimes d}$, $\alpha_k\in\mathbb{R}$, and $M$ is the number of the Pauli strings. Therefore, the expectation value of $O$ can be written as

\begin{align}
\langle \psi  | O | \psi \rangle
= \sum_{k=1}^M  \alpha_k \langle \psi  | P_k | \psi \rangle.
\end{align}

In the simplest way to estimate the expectation value $\langle \psi  | O |\psi \rangle$, each
of $\alpha_k \langle \psi  | P_k | \psi \rangle$ is measured on a quantum computer and then
assembled by classical postprocessing. However, for many applications, the number of quantities $M$ to be measured is $\mathcal{O}\left(\mathrm{poly}(d)\right)$, making the computation unfeasible if the system dimension $d$ is not small enough.
There are several studies to reduce the number of measurements to estimate expectation values of observables. Those works, To the best of our knowledge, are driven by: grouping Pauli strings, using classical shadows, and dealing with one part of the quantum state classically. Despite these efforts, the measurement counts for practical applications are not significantly reduced.

## 3.  Efficient quantum expectation values estimation for concentrated states

### 3.1 Motivation
Under the assumption that $|\psi \rangle$ is a concentrated state, it is defined a reduced state $|\psi_R \rangle$ as

\begin{align}
|\psi_R \rangle = \mathcal{N}_R \sum_{r=1}^{R} c_{i_r} |i_r \rangle,
\end{align}

where $R$ is a fixed number, $|c_{i_1}|^2 \geq |c_{i_2}|^2 \dots \geq |c_{i_R}|^2 \geq \max\{|c_{i}|^2: i \neq i_r \text{ for each } r=1\dots R\}$, and $\mathcal{N}_r = 1/\sqrt{ \sum_{r=1}^{R} |c_{i_r}|^2}$. Therefore, the expectation value of an observable $O$ acting on the system can be approximated by

\begin{align}
\langle \psi  | O | \psi \rangle \approx \mathcal{N}_R^2 \langle \psi_R  | O | \psi_R \rangle
= & \mathcal{N}_R^2 \sum_{r,r'=1}^R  c_{i_r} c_{i_r'}^* \, \langle i_{r'}  | O | i_r \rangle \\
= & \mathcal{N}_R^2 \left\{ \sum_{r=1}^R |c_{i_r}|^2 \langle i_r  | O | i_r \rangle + 2 \sum_{r < r'} \mathrm{Re}(c_{i_r} c_{i_r'}^*) \, \mathrm{Re} \langle i_{r'}  | O | i_r \rangle  -  2 \sum_{ r < r'} \mathrm{Im}(c_{i_r} c_{i_r'}^*) \, \mathrm{Im} \langle i_{r'}  | O | i_r \rangle \right\},
\end{align}

Here the cuantities $\langle i_{r'} |O | i_r \rangle$ can be calculated classically whenever the observable is a tensor product of one-qubit observable. Hence, by an estimation of the coefficients $|c_{i_r}|^2$ , $2\mathrm{Re}(c_{i_r} c_{i_r'}^*)$ and $2\mathrm{Im}(c_{i_r} c_{i_r'}^*)$ we can estimate the expectation value of $O$. For this purpose, we define a POVM $E$ composed of the elements
\begin{align}
P_k = K |k \rangle \langle k |, \quad 
D_{jk} = \frac{K}{2}(|j \rangle  + |k \rangle ) ( \langle  j| + \langle k |), \quad
A_{jk} = \frac{K}{2}(|j \rangle  - |k \rangle ) ( \langle  j| - \langle k |), \quad
L_{jk} = \frac{K}{2}(|j \rangle  + i|k \rangle ) ( \langle  j| -i \langle k |), \quad
R_{jk} = \frac{K}{2}(|j \rangle  - i|k \rangle ) ( \langle  j| + i \langle k |), 
\end{align}

and noticing that
 
\begin{align}
|c_{i}|^2 =   \, \frac{\mathrm{Tr}\left(\rho P_{j} \right)}{K},\qquad
2 \mathrm{Re}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho D_{jk} \right) - \mathrm{Tr}\left(\rho A_{jk} \right)}{K},\qquad
-2 \mathrm{Im}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho L_{jk} \right) - \mathrm{Tr}\left(\rho R_{jk} \right)}{K},
\end{align}

we approximate $\langle \psi  | O | \psi \rangle$ by measuring the POVM $E$. 

$\textbf{Observation:}$ Since concentrate states usually present interactions for a small number of bodies,  there are some quantities $\langle i_{r'}  | O | i_r \rangle = 0 $. Therefore, some coefficients are negligible due to the structure of the quantum state. This reduction of coefficients improves the required measurement resources to compute the approximation, hence improving the estimation of $\langle \psi  | O | \psi \rangle$.


### 3.2 The method

To implement the above idea we reproduce the following procedure:

i) Prepare the state $|\psi\rangle$ on quantum computers, and measure it in the computational basis $\{| i \rangle : i = 0,1,2^n-1\}$ with a number $L_r$ of shoots.

ii) From the outcomes of the measurement we  pick up the most frequent R elements such that $1-|\langle \psi_R | \psi \rangle|^2$ be lower of some previusly defined tolerance $Tol$ ($Tol\approx 10^{-4}$). Next, we sort in descending order of frecuency the selected elements.

iii) Determine the number of body interactions for the system to reduce the number of coefficients in the estimation.

iv) Measure The POVM $E$ and compute the non-negligible coefficientes $|c_{i_r}|^2$ , $2\mathrm{Re}(c_{i_r} c_{i_r'}^*)$ and $2\mathrm{Im}(c_{i_r} c_{i_r'}^*)$ as a sample mean of the outcomes.

The described method inherits the performance properties shown in (cita paper). First, the error in estimating  $\langle \psi  | O | \psi \rangle$ produced by  approximating  the quantum estate by $| \psi_R\rangle$ is proportional to $\sqrt{1-|\langle \psi_R | \psi \rangle|^2}$. Next, the number of measurements necessary to attain some precision in the standard deviation of the approximation is fewer than that of the alternative techniques. Moreover, the described method improves to that shown in (cita paper), since the number of CNOT gates employed is not $\mathcal{O}(d)$, but is equal to the maximum number of body interactions.

## 4. Implementation using Qiskit


## 5. VQE method for molecular hamiltonians 

## 7. Discussion and outlook

## References
